# Minimizing the Travel Distance of Pharmacy Students to Hospitals


## Introduction
   ### Each semester [TSU College of Pharmacy](http://www.tsu.edu/academics/colleges-and-schools/college-of-pharmacy-and-health-sciences/) sends graduate pharmacy students to Houston hospitals as a part of their internship program. The pharmacy program on average assigns more than three hundred students to different available locations across Houston. Although the program administration spends a long time to find the right location for each student, sometimes many students are asisgned to the locations far from their home.
 ### The ojective of this notebook is to offer a solution for minimization of the total distance that students travel to reach to the hospital while avoiding long commutes. To solve the problem, deap, a python library that include a genetic algorithm code, is used.
 ### The notebook workflow is as follows:
 * Import required libraries
 * Read csv files
 * Data wrangling
 * Create deap model
 * Create an objective function
 * Assign the objective function to the model
 * Run the model
 
### The full procedure could be found below.

In [1]:
### Uncomment any missing library

#! pip install --user numpy
#! pip install --user pandas
#! pip install --user deap

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import random
from deap import algorithms, base, creator, tools

In [3]:
# Read a csv file containing lat and lon of city of Houston zip codes 
df_ZipCodeCoord = pd.read_csv('City_of_Houston_Zip_Codes.csv',index_col=0)
df_ZipCodeCoord.head

<bound method NDFrame.head of      Zip_Code   Latitude  Longitude
0       77001  29.770000 -95.370000
1       77002  29.756845 -95.365652
2       77003  29.749778 -95.345885
3       77004  29.724893 -95.363752
4       77005  29.718435 -95.423555
..        ...        ...        ...
297     77488  29.307215 -96.091291
298     77489  29.600434 -95.515549
299     77493  29.853215 -95.831456
300     77494  29.740677 -95.829652
301     77498  29.643591 -95.653255

[302 rows x 3 columns]>

In [4]:
# Change the index to zip code
df_ZipCodeCoord = df_ZipCodeCoord.set_index('Zip_Code')
df_ZipCodeCoord

,Latitude,Longitude
Zip_Code,,
77001,29.770000,-95.370000
77002,29.756845,-95.365652
77003,29.749778,-95.345885
77004,29.724893,-95.363752
77005,29.718435,-95.423555
...,...,...
77488,29.307215,-96.091291
77489,29.600434,-95.515549
77493,29.853215,-95.831456


In [5]:
# Create a dictionary where Dict['Latitude'][Zip_code] returns the Lat of zip code
df_ZipCodeCoord_dict = df_ZipCodeCoord.to_dict()
df_ZipCodeCoord_dict

{'Latitude': {77001: 29.77,
  77002: 29.756845,
  77003: 29.749778,
  77004: 29.724893,
  77005: 29.718435,
  77006: 29.74097,
  77007: 29.771545,
  77008: 29.798249,
  77009: 29.795344,
  77010: 29.753624,
  77011: 29.743217,
  77012: 29.718525,
  77013: 29.795268,
  77014: 29.981209,
  77015: 29.76393,
  77016: 29.862532,
  77017: 29.689824,
  77018: 29.826448,
  77019: 29.75415,
  77020: 29.773179,
  77021: 29.69843,
  77022: 29.83159,
  77023: 29.721825,
  77024: 29.772179,
  77025: 29.685706,
  77026: 29.800187,
  77027: 29.740079,
  77028: 29.827869,
  77029: 29.759665,
  77030: 29.706787,
  77031: 29.652205,
  77032: 29.987805,
  77033: 29.66688,
  77034: 29.61951,
  77035: 29.655503,
  77036: 29.701847,
  77037: 29.89036,
  77038: 29.918595,
  77039: 29.911171,
  77040: 29.874575,
  77041: 29.8586,
  77042: 29.741325,
  77043: 29.81093,
  77044: 29.906616,
  77045: 29.635793,
  77046: 29.733777,
  77047: 29.61065,
  77048: 29.618714,
  77049: 29.832928,
  77050: 29.902887,
  77

In [6]:
# Read csv file containing the zip code of the students
df_students = pd.read_csv('Students_Zip_Code.csv',index_col=0)
df_students.head

<bound method NDFrame.head of     Students_Zip_Code
0               77007
1               77015
2               77016
3               77019
4               77022
..                ...
95              77089
96              77061
97              77066
98              77079
99              77029

[100 rows x 1 columns]>

In [7]:
# Read csv file containing the zip code of the hospitals and their capacity
df_Hospitals = pd.read_csv('Hospitals_Zip_Code.csv',index_col=0)
df_Hospitals.head

<bound method NDFrame.head of    Hospital_Zip_Code  Capacity
0              77022        14
1              77025         4
2              77088         4
3              77012         9
4              77055        21
5              77001        12
6              77009         8
7              77346         4
8              77253        16
9              77096         8>

In [8]:
# Create a dataframe that contains the zip code of every position
list_zip_hosp = []
for index, row in df_Hospitals.iterrows():
    for cap in range(row['Capacity']):
        list_zip_hosp.append(row['Hospital_Zip_Code'])

df_Hospitals_extended = pd.DataFrame(list_zip_hosp, columns =['Hosp_Zip_codes'])

In [9]:
# Creates a new class named "FitnessMin" inheriting from "base.Fitness" with attrebute "weights=(-1.0,)"
# The fitness is a measure of quality of a solution.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # -1 -> minimum problem
creator.create("Individual", list, fitness=creator.FitnessMin)

In [10]:
# Create a toolbox
toolbox = base.Toolbox()

In [11]:
# Attribute generator 
toolbox.register("index", np.random.choice, len(df_students), len(df_students), replace=False) # choose all spots

In [12]:
# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.index)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [13]:
# Sample individual 
ind = toolbox.individual()
ind

[92,
 57,
 90,
 27,
 17,
 10,
 71,
 86,
 76,
 46,
 38,
 79,
 9,
 50,
 42,
 99,
 19,
 65,
 26,
 30,
 48,
 25,
 40,
 13,
 51,
 97,
 60,
 35,
 43,
 93,
 16,
 95,
 20,
 78,
 82,
 41,
 67,
 45,
 1,
 28,
 49,
 52,
 4,
 94,
 29,
 96,
 58,
 89,
 47,
 8,
 59,
 84,
 44,
 74,
 80,
 83,
 31,
 54,
 22,
 5,
 39,
 91,
 98,
 34,
 75,
 32,
 2,
 23,
 33,
 88,
 73,
 66,
 0,
 18,
 55,
 64,
 3,
 11,
 15,
 14,
 81,
 7,
 69,
 87,
 56,
 12,
 63,
 85,
 61,
 6,
 62,
 72,
 77,
 53,
 68,
 70,
 21,
 24,
 36,
 37]

In [14]:
## Function for get a total distance of student travel
def total_distance(ind):
    
    df_students_temp = df_students.iloc[ind]
    
    dist_sum = sum(distance(df_students_temp.iloc[i]['Students_Zip_Code'], df_Hospitals_extended.iloc[i]['Hosp_Zip_codes']) for i in range(df_students_temp.shape[0]))
    
    return dist_sum

In [15]:
def distance(zip1, zip2):
    # import lat and lon of zip codes, both students and hospitals
    lng1, lat1, lng2, lat2 = radians(df_ZipCodeCoord_dict['Longitude'][zip1]), radians(df_ZipCodeCoord_dict['Latitude'][zip1]), radians(df_ZipCodeCoord_dict['Longitude'][zip2]), radians(df_ZipCodeCoord_dict['Latitude'][zip2])
    
    # FAA approved globe radius in km (radius of the earth)
    RADIUS = 6371 
    
    dlng = lng2-lng1
    dlat = lat2-lat1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlng/2)**2
    c = 2 * asin(sqrt(a)) 
    
    dist = RADIUS * c
    
    # Multiply by distance to penalize long commutes
    return dist * dist

In [16]:
def eval_func(individual):
    
    # 1 total distance -> minimun
    t_dist = total_distance(individual)
    
    # 2 penalty
    penalty = len(individual) - len(set(individual))
    t_dist += penalty*1000000
    
    return t_dist,

In [17]:
# Test eval function
eval_func(ind)

(48542.18590566115,)

In [18]:
# Add eval funcion to toolbox
toolbox.register("evaluate", eval_func)

In [19]:
# Create the optimization requiremenst
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxTwoPoint)
# tools.mutShuffleIndexes : Shuffle the attributes of the input individual and return the mutant.
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.8)
hof = tools.HallOfFame(1)

In [20]:
# Optimization parameters
POP_SIZE = 200
MAX_GEN = 100
MUT_PROB = 0.2
CX_PROB = 0.8

In [21]:
# Example of generated population
pop = toolbox.population(n=POP_SIZE)
pop

[[16,
  47,
  41,
  79,
  86,
  72,
  2,
  1,
  10,
  42,
  61,
  85,
  40,
  62,
  17,
  52,
  81,
  7,
  39,
  46,
  92,
  21,
  37,
  73,
  9,
  53,
  66,
  55,
  35,
  25,
  90,
  49,
  84,
  64,
  56,
  89,
  74,
  27,
  68,
  78,
  34,
  12,
  20,
  36,
  29,
  19,
  91,
  95,
  31,
  13,
  69,
  4,
  67,
  60,
  63,
  98,
  82,
  51,
  77,
  6,
  87,
  48,
  75,
  0,
  94,
  44,
  70,
  26,
  71,
  80,
  45,
  22,
  57,
  50,
  59,
  14,
  15,
  88,
  33,
  11,
  30,
  58,
  5,
  99,
  96,
  18,
  83,
  38,
  93,
  3,
  28,
  54,
  43,
  8,
  76,
  65,
  97,
  23,
  32,
  24],
 [28,
  52,
  25,
  38,
  37,
  7,
  39,
  53,
  58,
  21,
  96,
  78,
  42,
  91,
  68,
  79,
  13,
  74,
  17,
  61,
  99,
  30,
  12,
  98,
  90,
  83,
  50,
  45,
  60,
  5,
  18,
  14,
  97,
  47,
  89,
  26,
  56,
  1,
  36,
  27,
  65,
  51,
  82,
  0,
  64,
  43,
  87,
  85,
  81,
  16,
  19,
  62,
  70,
  20,
  66,
  31,
  69,
  24,
  49,
  55,
  84,
  86,
  72,
  22,
  8,
  71,
  88,
  46,
  3,
 

In [22]:
# Add statistical information to the output
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0) 
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

In [25]:
%%time 
# Run the case
result, log = algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=POP_SIZE, # The number of individuals to select for the next generation.
                                     lambda_= POP_SIZE, # The number of children to produce at each generation.
                                     cxpb= CX_PROB,
                                     mutpb= MUT_PROB, 
                                     halloffame=hof,
                                     stats= stats, 
                                     ngen= MAX_GEN,
                                     verbose= True)

gen	nevals	avg             	min             	max             
0  	200   	[47093.76018826]	[39869.74715143]	[54324.87796214]
1  	200   	[46428.29510091]	[39869.74715143]	[49736.00695606]
2  	200   	[45866.91303632]	[39869.74715143]	[48570.72319301]
3  	200   	[45505.32718869]	[39869.74715143]	[47828.15564415]
4  	200   	[45311.69077079]	[39869.74715143]	[47554.51020149]
5  	200   	[45068.35538715]	[39869.74715143]	[47182.12586764]
6  	200   	[44904.69644284]	[39869.74715143]	[46783.39252416]
7  	200   	[44671.89460244]	[39869.74715143]	[46533.1472895] 
8  	200   	[44465.1495567] 	[39816.19501853]	[46293.28675968]
9  	200   	[44367.23563997]	[39816.19501853]	[46156.78521387]
10 	200   	[44193.23339648]	[39816.19501853]	[46002.42010578]
11 	200   	[44014.80305782]	[39816.19501853]	[45831.15020379]
12 	200   	[43903.07246267]	[39816.19501853]	[45716.08688868]
13 	200   	[43750.79499919]	[39816.19501853]	[45568.1005261] 
14 	200   	[43629.71111891]	[39816.19501853]	[45405.36554799]
15 	200 

In [ ]:
result

In [26]:
hof